In [6]:

import pprint
import sys
sys.path.append('/Users/obara/work/prescription')

In [11]:
preprocessing(upload_files)

({'top_left_1': '',
  'top_left_2': '',
  'top_right_1': '',
  'top_right_2': '',
  'med': '',
  'insurance_patient_num': [['番号', 'o 6271340']]},
 False)

In [10]:
#coding:utf-8
#必要なもののimport 
import base64
import json
from requests import Request, Session
from io import BytesIO
from PIL import Image
import pandas as pd
import jaconv
import re
import cv2
import numpy as np
from symspellpy.symspellpy import SymSpell, Verbosity
import pickle
from pykakasi import kakasi
import Levenshtein
from collections import OrderedDict
import itertools
import os
import time

#別ファイルの読み込み
# import basic_info as bi
# import med_info as mi
import others as others
import qrcode as qr

#------------------------------
#前処理部分全体　

url="/Users/obara/PHC.png"
upload_files=[1]
def preprocessing(upload_files):
    '''
    input:upload_files
    output:OCR結果を入れた辞書{"top_left_1":"","top_left_2":"","top_right_1":"","top_right_2":"","med":"","insurance_patient_num":""},ボケ具合
    '''
    #枚数分だけfor文を回す
    ocr_result={"top_left_1":"","top_left_2":"","top_right_1":"","top_right_2":"","med":"","insurance_patient_num":""}
    for file_num in range(len(upload_files)):
#         file = upload_files[file_num]
        filename = str(time.time()) + ".png"
#         file.save(os.path.join('./uploads', filename))

        #image = others.path_to_array("uploads/" + filename)
        image = others.path_to_array(url)
        #ボケ度を数値化（True/False)
        boke = boke_check(image)
        #歪み補正
        image_yugami_finished = yugami(image)
        #タイプを判別　Aが一段組、Bが2段組
        prescription_shape_type = type_check(image_yugami_finished)
        #切り取り
        #[0]基本情報部分、[1]基本情報の左上、[2]基本情報の左下、[3]基本情報の右上、[4]基本情報の右下、[5]処方情報部分、[6]処方情報の左、[7]処方情報の右、[8]保険者番号部分、[9]PHCパラメータで保険者番号を切り取った場合、[10]矢澤先生パラメータで保険者番号を切り取った場合、[11]記号番号
        image_cut_finished = cut(image)


        #1枚目
        if file_num == 0:
            #処方箋タイプがA（一段組）の時
            if prescription_shape_type=="A":
                kinds=[image_cut_finished[1],image_cut_finished[2],image_cut_finished[3],image_cut_finished[4],image_cut_finished[5]]
                result_dic = others.api_pararel(kinds)
                med_text = result_dic["bottom_first"]
            
            #処方箋タイプがB（2段組）の時
            else:
                kinds = [image_cut_finished[1], image_cut_finished[2], image_cut_finished[3], image_cut_finished[4], image_cut_finished[6], image_cut_finished[7]]
                result_dic = others.api_pararel(kinds)
                med_text=result_dic["bottom_first"]+result_dic["bottom_second"]
            
            ocr_result["top_left_1"] = result_dic["top_left_1"]
            ocr_result["top_left_2"] = result_dic["top_left_2"]
            ocr_result["top_right_1"] = result_dic["top_right_1"]
            ocr_result["top_right_2"] = result_dic["top_right_2"]
            ocr_result["insurance_patient_num"]=others.recognize2(image_cut_finished[8])
            ocr_result["med"]=med_text
            
        #2枚目以降
        else:
            #処方箋タイプがA（一段組）の時
            if prescription_shape_type == "A":
                med_text = others.recognize2(image_cut_finished[5])
            #処方箋タイプがB（2段組）の時
            else:
                #本当はここも並列化できるけれど、割愛
                med_text = others.recognize2(image_cut_finished[6]) + others.recognize2(image_cut_finished[7])
            ocr_result["med"]=ocr_result["med"]+med_text

    return ocr_result,boke
    



#------------------------------
ALLOWED_EXTENSIONS = set(['png','jpg','jpeg',])
def allwed_file(filename):
    ''' .があるかどうかのチェックと、拡張子の確認
    # OKなら１、だめなら0'''
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


#------------------------------
#ピンボケ検出
def boke(image):
    '''ピンボケ度を数値で返す
    input:array
    ooutput:数値
    '''
    return cv2.Laplacian(image, cv2.CV_64F).var()

def boke_check(image):
    '''ピンボケ度の数値を閾値で評価
    input:array
    output:True/False
    '''
    if boke(image)<200:
        return True
    else:
        return False


#------------------------------
#輪郭を検出する。
def waku_detection(img):
    ''' 
    input:画像のarray
    output:元の画像のarray,補正前の4隅の座標,補正前の横の長さ
    '''
    #画像をグレースケール化
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    #閾値を180にして2値化
    threshold = 105
    ret, img_thresh = cv2.threshold(img_gray, threshold, 255, cv2.THRESH_BINARY)

    #輪郭を取り出している
    img_1, contours, hierarchy = cv2.findContours(img_thresh , cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #mensekiリストに輪郭を追加していっている。
    menseki=[ ]

    for i in range(0, len(contours)):
        menseki.append([contours[i],cv2.contourArea(contours[i])])

    menseki.sort(key=lambda x: x[1], reverse=True)

    #一番面積が大きいものを取り出している。
    cnt = menseki[0][0]

    #輪郭のギザギザを無くしている
    epsilon = 0.1*cv2.arcLength(cnt,True)
    #approxに隅の四点の座標が入っている
    approx = cv2.approxPolyDP(cnt,epsilon,True)

    #ギザギザをなくした後の描画
    #img3=cv2.drawContours(img, approx, 0,(0, 0, 255),10)

    #輪郭の表をリストにして、順番を整理
    approx=approx.tolist()

    left = sorted(approx,key=lambda x:x[0]) [:2]
    right = sorted(approx, key=lambda x: x[0])[2:]
    
    left_down= sorted(left,key=lambda x:x[0][1]) [0]
    left_up= sorted(left,key=lambda x:x[0][1]) [1]

    right_down= sorted(right,key=lambda x:x[0][1]) [0]
    right_up= sorted(right,key=lambda x:x[0][1]) [1]

    #補正前の角の座標
    perspective1 = np.float32([left_down, right_down, right_up, left_up])
    #補正後の横の長さ
    width = right_down[0][0] - left_down[0][0]
    return img,perspective1,width

#------------------------------
def yugami(image_array):
    '''歪み補正
    input:画像のPath
    output:歪み補正した画像のarray
    '''
    #枠検出した結果をwaku_detection_resultに格納
    waku_detection_result=waku_detection(image_array)
    img = waku_detection_result[0]
    perspective1 = waku_detection_result[1]
    width = waku_detection_result[2]
    #補正後の縦の長さ
    height=width*2340//1654
    
    perspective2 = np.float32([[0, 0],[width, 0],[width, height],[0, height]])

    #変換に必要な行列
    psp_matrix = cv2.getPerspectiveTransform(perspective1,perspective2)
    #変換後
    #img_psp = cv2.warpPerspective(img, psp_matrix, (1654, 2340))
    img_psp = cv2.warpPerspective(img, psp_matrix, (width, height))
    return img_psp


#------------------------------
def type_check(img):
    '''
    2段組かどうかを判別
    input:画像のarray（歪み補正後）
    output:A（1段組）or B（2段組）
    '''
    height=img.shape[0]
    width=img.shape[1]

    img=img[height//2: 5 * height // 8, width*2//5:width*3//5]
    #img = cv2.resize(img,(int(img.shape[1]/5),int(img.shape[0]/5)))
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #gray = cv2.GaussianBlur(gray,(5,5),5)
    
#     plt.gray()
#     plt.imshow(gray)

    edges = cv2.Canny(gray,50,150,apertureSize = 3)

    linesH = cv2.HoughLinesP(edges, rho=1, theta=np.pi / 360, threshold=50, minLineLength=50, maxLineGap=10)
    try:
        if len(linesH)!=0:
            for line in linesH:
                x1, y1, x2, y2 = line[0]
                if (x1-x2)**2<(y1-y2)**2:
                    return "B"
                else:
                    return "A"
            
        else:
            return "A"
    except:
        return "A"

def cut(image):
    '''
    画像を領域ごとに切り取る
    input:画像のarray
    output:画像のarrayのリスト
    [0]基本情報部分、[1]基本情報の左上、[2]基本情報の左下、[3]基本情報の右上、[4]基本情報の右下、[5]処方情報部分、[6]処方情報の左、[7]処方情報の右、[8]保険者番号部分、[9]PHCパラメータで保険者番号を切り取った場合、[10]矢澤先生パラメータで保険者番号を切り取った場合、[11]記号番号
    '''
    
    #画像をグレースケール化 入れるのはarray
    img_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    #閾値を180にして2値化
    # threshold = 10
    # ret, img_thresh = cv2.threshold(img_gray, threshold, 255, cv2.THRESH_BINARY)

    #2値化後を表示
#     plt.imshow( img_gray)
#     plt.gray()

    height=img_gray.shape[0]
    width=img_gray.shape[1]
    #切り出し
    img2=image[0:612*height//2340,width//2:width]              
                     
    # BGR -> グレースケール
    gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # エッジ抽出 (Canny)
    edges = cv2.Canny(gray, 50, 190, apertureSize=3)
    #cv2.imwrite('wakuedges.png', edges)
    # 膨張処理
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    edges = cv2.dilate(edges, kernel)


    # 輪郭抽出
    _, contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # 面積でフィルタリング
    rects = []
    mensekilist=[]
    for cnt, hrchy in zip(contours, hierarchy[0]):
        if cv2.contourArea(cnt) < 30:
            continue  # 面積が小さいものは除く
        if hrchy[3] == -1:
            continue  # ルートノードは除く
        # 輪郭を囲む長方形を計算する。
        rect = cv2.minAreaRect(cnt)
        rect_points = cv2.boxPoints(rect).astype(int)
        rects.append(rect_points)
        mensekilist.append(cv2.contourArea(cnt) )

    indexnum=mensekilist.index(max(mensekilist))

    # x-y 順でソート
    #rects = sorted(rects, key=lambda x: (x[0][1], x[0][0]))

    # 描画する。
    # for i, rect in enumerate(rects):
    #     color = np.random.randint(0, 255, 3).tolist()
        #cv2.drawContours(img, rects, i, color, 2)
        #cv2.putText(img, str(i), tuple(rect[0]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 3)

        #print('rect:\n', rect)

    i=indexnum
    x=[rects[i][0][0],rects[i][1][0],rects[i][2][0],rects[i][3][0]]
    x1=sorted(x)
    y=[rects[i][0][1],rects[i][1][1],rects[i][2][1],rects[i][3][1]]
    y1=sorted(y)
    smallx=round((x1[1] +x1[0])//2)
    largex=round((x1[-1]+x1[-2])//2)
    smally=round((y1[0]+y1[1])//2)
    largey=round((y1[-1]+y1[-2])//2)
    sitanum=img2[smally:largey,smallx:largex]
    #cv2.imwrite('sitanum2/'+str(num)+'.png', sitanum)

    #以下PHCverのパラメータ       
    img_list=[]
    for i in range(8):
        n=i
        height1=largey-smally
        width1=int(round((largex-smallx)/8))
        difference=int(round((height1-width1)/3))
        difference2=width1//7

        if n<4:
            img3=img2[smally-height1+3*difference:largey-height1-difference,(smallx+width1*n)+0*difference2:smallx+width1*(n+1)-2*difference2]

        else:
            img3=img2[smally-height1+3*difference:largey-height1-difference,(largex-width1*(8-n))+0*difference2:largex-width1*(7-n)-2*difference2]
        img_list.append(img3)

    im_h = cv2.hconcat(img_list)

    #以下矢澤先生verのパラメータ
    img_list2=[]
    for i in range(8):
        n=i
        height1=largey-smally
        width1=int(round((largex-smallx)/8))
        difference=int(round((height1-width1)/3))
        difference2=width1//7

        if n<4:
            img3=img2[smally-height1+1*difference:largey-height1-difference,(smallx+width1*n)+1*difference2:smallx+width1*(n+1)-0*difference2]

        else:
            img3=img2[smally-height1+1*difference:largey-height1-difference,(largex-width1*(8-n))+1*difference2:largex-width1*(7-n)-0*difference2]
        img_list2.append(img3)

    im_h2 = cv2.hconcat(img_list2)


    #記号番号の上のところを取得する
    img4 = img2[smally - height1 + 3 * difference : largey - height1 - difference, smallx:largex]
    
    #矢澤先生の処方箋の時
    # top = image[0: 900 * height // 2340, 0 :width]
    # bottom1 = image[900 * height // 2340 :height, 0 :width//2]
    # bottom2 = image[900 * height // 2340 :height, width // 2 :width]
    # bottom_all=image[900 * height // 2340 :height, 0 :width]

    #PHCの処方箋の時・・・topを少し広めに取り、境界領域の情報を掬えるようにする
    top = image[0:400 * height // 1030, 0:width]
    top_left_1 = image[0:200 * height // 1030, 0:width // 2]
    top_left_2 = image[200 * height // 1030:400 * height // 1030, 0:width]
    top_right_1 = image[0:200 * height // 1030, width//2:width]
    top_right_2 = image[200 * height // 1030:400 * height // 1030, width//2:width]
    
    bottom1 = image[300 * height // 1030 :height, 0 :width//2]
    bottom2 = image[300 * height // 1030 :height, width // 2 :width]
    bottom_all = image[300 * height // 1030 :height, 0:width]


    #[0]基本情報部分、[1]基本情報の左上、[2]基本情報の左下、[3]基本情報の右上、[4]基本情報の右下、[5]処方情報部分、[6]処方情報の左、[7]処方情報の右、[8]保険者番号部分、[9]PHCパラメータで保険者番号を切り取った場合、[10]矢澤先生パラメータで保険者番号を切り取った場合、[11]記号番号
    return top,top_left_1,top_left_2,top_right_1,top_right_2,bottom_all,bottom1,bottom2,img4,im_h,im_h2, sitanum